In [1]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras import optimizers

import datetime
import numpy as np

Using TensorFlow backend.


In [2]:
def get_timestamp():
    return '{:%Y-%m-%d-%H-%M-%S}'.format(datetime.datetime.now())


def get_tensorboard(name=None):
    name = name or 'Graph'
    return TensorBoard(
        log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
        write_graph=True, write_images=False
    )

In [3]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [4]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(cifar10)

In [5]:
datagen = ImageDataGenerator(width_shift_range=.2,
                            height_shift_range=.25,
                            horizontal_flip=True)
datagen.fit(x_train)

In [6]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
linear = Sequential()
linear.add(Flatten(input_shape=x_train.shape[1:]))
linear.add(Dense(units=10, activation='softmax'))

In [8]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
small_cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
small_cnn.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
small_cnn.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Flatten())
small_cnn.add(Dense(units=100, activation='relu'))
small_cnn.add(Dense(units=50, activation='relu'))

small_cnn.add(Dense(units=10, activation='softmax'))

In [9]:
#model = linear
model = small_cnn


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers.adagrad(),
              metrics=['accuracy'])

In [ ]:
n_train    = 6000
batch_size = 128
epochs     = 10

tbCallBack = get_tensorboard()
model.fit_generator(datagen.flow(x_train[:n_train], y_train[:n_train], batch_size=batch_size),
                    steps_per_epoch=len(x_train[:n_train]) / batch_size, epochs=epochs,
                    validation_data=(x_test[:n_train], y_test[:n_train]),
                    callbacks=[tbCallBack])

Epoch 1/10
47/46 [==============================] - 41s - loss: 2.5632 - acc: 0.1058 - val_loss: 2.3026 - val_acc: 0.1040
Epoch 2/10
47/46 [==============================] - 40s - loss: 2.3028 - acc: 0.1017 - val_loss: 2.3026 - val_acc: 0.1005
Epoch 3/10
47/46 [==============================] - 38s - loss: 2.3133 - acc: 0.1098 - val_loss: 2.3026 - val_acc: 0.1005
Epoch 4/10
47/46 [==============================] - 42s - loss: 2.3148 - acc: 0.1167 - val_loss: 2.3028 - val_acc: 0.1005
Epoch 5/10
47/46 [==============================] - 40s - loss: 2.3029 - acc: 0.1024 - val_loss: 2.3023 - val_acc: 0.1153
Epoch 6/10
47/46 [==============================] - 40s - loss: 2.3013 - acc: 0.1153 - val_loss: 2.2922 - val_acc: 0.1020
Epoch 7/10
47/46 [==============================] - 41s - loss: 2.2579 - acc: 0.1533 - val_loss: 2.1479 - val_acc: 0.1990
Epoch 8/10
38/46 [=======================>......] - ETA: 5s - loss: 2.1455 - acc: 0.1951

In [ ]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', acc)